# Ring plot

Based on the code provided here: https://doi.org/10.5281/zenodo.7016057 and simplified to not use ellipticity. Huge thanks to Max Isi for this helpful code.

In [ ]:
import matplotlib as mpl
from matplotlib import transforms
from matplotlib.path import Path
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from thesis_utils.plotting import save_figure

plt.rcParams["text.usetex"] = True

In [ ]:
class EllipticalWedge(mpl.patches.Patch):
    """Wedge shaped patch."""

    def __str__(self):
        pars = (
            self.center[0],
            self.center[1],
            self.r,
            self.theta1,
            self.theta2,
            self.width,
        )
        fmt = "Wedge(center=(%g, %g), r=%g, theta1=%g, theta2=%g, width=%s)"
        return fmt % pars

    def __init__(
        self, center, r, theta1, theta2, ellip=1, angle=0, width=None, **kwargs
    ):
        """
        A wedge centered at *x*, *y* center with radius *r* that
        sweeps *theta1* to *theta2* (in degrees).  If *width* is given,
        then a partial wedge is drawn from inner radius *r* - *width*
        to outer radius *r*.
        Valid keyword arguments are:
        %(Patch_kwdoc)s
        """
        super().__init__(**kwargs)
        self.center = center
        self.r, self.width = r, width
        self.theta1, self.theta2 = theta1, theta2
        self.ellip = ellip
        self.angle = angle
        self._patch_transform = transforms.IdentityTransform()
        self._recompute_path()

    def _recompute_path(self):
        # Inner and outer rings are connected unless the annulus is complete
        if abs((self.theta2 - self.theta1) - 360) <= 1e-12:
            theta1, theta2 = 0, 360
            connector = Path.MOVETO
        else:
            theta1, theta2 = self.theta1, self.theta2
            connector = Path.LINETO

        # Form the outer ring
        arc = Path.arc(theta1, theta2)

        if self.width is not None:
            # Partial annulus needs to draw the outer ring
            # followed by a reversed and scaled inner ring
            v1 = arc.vertices
            v2 = arc.vertices[::-1] * (self.r - self.width) / self.r
            v = np.concatenate([v1, v2, [v1[0, :], (0, 0)]])
            c = np.concatenate(
                [arc.codes, arc.codes, [connector, Path.CLOSEPOLY]]
            )
            c[len(arc.codes)] = connector
        else:
            # Wedge doesn't need an inner ring
            v = np.concatenate(
                [arc.vertices, [(0, 0), arc.vertices[0, :], (0, 0)]]
            )
            c = np.concatenate(
                [arc.codes, [connector, connector, Path.CLOSEPOLY]]
            )

        # squeeze and rotate
        v[:, 1] *= self.ellip

        costh = np.cos(self.angle)
        sinth = np.sin(self.angle)
        R = np.array([[costh, -sinth], [sinth, costh]])
        v = np.dot(R, v.T).T

        # Shift and scale the wedge to the final location.
        v *= self.r
        v += np.asarray(self.center)

        self._path = mpl.path.Path(v, c)

    def set_center(self, center):
        self._path = None
        self.center = center
        self.stale = True

    def set_radius(self, radius):
        self._path = None
        self.r = radius
        self.stale = True

    def set_theta1(self, theta1):
        self._path = None
        self.theta1 = theta1
        self.stale = True

    def set_theta2(self, theta2):
        self._path = None
        self.theta2 = theta2
        self.stale = True

    def set_width(self, width):
        self._path = None
        self.width = width
        self.stale = True

    def get_path(self):
        if self._path is None:
            self._recompute_path()
        return self._path

In [ ]:
# Basis vectors in 2D
ep = np.array([[1, 0], [0, -1]])
ec = np.array([[0, 1], [1, 0]])

In [ ]:
def polarizations(wt, A=1.0, theta=0):
    """Value of the two polarizations"""
    hp = np.cos(wt) * np.cos(theta) - np.sin(wt) * np.sin(theta)
    hc = np.cos(wt) * np.sin(theta) + np.sin(wt) * np.cos(theta)
    return A * hp, A * hc

In [ ]:
def displacement(coords, wt, A=1.0, theta=0):
    """Displacement caused by the plus and cross polarizations"""
    hp, hc = polarizations(wt, A=A, theta=theta)
    return np.dot(hp * ep + hc * ec, coords)

In [ ]:
def plot_diagram(
    A=0.2,
    A0=1,
    Nt=5,
    Npoints=16,
    ax_width=None,
    i0=4,
    palette="crest",
    phasor=False,
):
    wts = np.linspace(0, 2 * np.pi, Nt)

    th = np.linspace(0, 2 * np.pi, Npoints, endpoint=False)
    X = A0 * np.cos(th)
    Y = A0 * np.sin(th)

    ax_width = ax_width or 2 * A0

    nrow = 2
    if phasor:
        nrow += 1
    fig = plt.figure(
        constrained_layout=True, figsize=(ax_width * Nt, nrow * ax_width)
    )
    gs = fig.add_gridspec(nrow, Nt)

    # draw the trajectory of a highlighted point
    x, y = X[i0], Y[i0]
    dxs, dys = [], []
    for wt in np.linspace(0, 2 * np.pi, 50):
        dx, dy = displacement([x, y], wt, A=A)
        dxs.append(dx)
        dys.append(dy)
    dxs = np.array(dxs)
    dys = np.array(dys)

    # initial angle for highlighted point
    th0 = np.arctan2(dys[0], dxs[0])

    c = sns.color_palette(palette, n_colors=Npoints)
    # c = sns.cubehelix_palette(start=.5, rot=-.75, n_colors=Npoints)
    for i, wt in enumerate(wts):
        ax = fig.add_subplot(gs[0, i])

        ax.axhline(0, 0.5, 0.6, c="gray", lw=0.5)
        ax.axvline(0, 0.5, 0.6, c="gray", lw=0.5)
        kws = dict(
            fontsize=14, horizontalalignment="right", verticalalignment="top"
        )
        if i == 0:
            ax.annotate(r"$x$", xy=(0.5, 0), **kws)
            ax.annotate(r"$y$", xy=(-0.1, 0.5), **kws)

        dX, dY = displacement([X, Y], wt, A=A)

        # highlighted point
        wedge = EllipticalWedge(
            [X[i0], Y[i0]],
            A * A0,
            0,
            np.rad2deg(wt),
            angle=th0,
            color=c[i0],
            alpha=0.1,
        )
        ax.add_patch(wedge)
        ax.annotate(
            "",
            xy=(X[i0] + dX[i0], Y[i0] + dY[i0]),
            xytext=(X[i0], Y[i0]),
            arrowprops=dict(arrowstyle="-", color=c[i0], alpha=0.5),
        )
        ax.plot(x + dxs, y + dys, c=c[i0], alpha=0.5, lw=1, ls=":")

        ax.scatter(
            X, Y, label=wt, edgecolors=c, facecolor="none", s=5, alpha=0.5
        )
        ax.scatter(X + dX, Y + dY, c=c)

        ax.axis("off")
        ax.axis("equal")
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)

    # get offset of first subplot center in display coordinates
    w = ax.transData.transform((-1.2, 0))[0]

    ax = fig.add_subplot(gs[1, :])

    # convert the offset to coordinates of the new axis in
    w_new = 0.85 * ax.transData.inverted().transform((w, 0))[0]

    wts2 = np.linspace(0, 2 * np.pi, 100)
    ax.axhline(0, c="gray", ls=":")  # c='0.8', lw=3, ls='--')
    for wt in wts:
        ax.axvline(wt, 0, 1, c="gray", ls=":")

    hp2, hc2 = polarizations(wts2, A=1)

    ax.plot(wts2, hp2, lw=2, c=c[-1], label=r"$+$")
    ax.plot(wts2, hc2, lw=2, c=c[-5], ls="--", label=r"$\times$")

    ax.legend(loc="lower right", edgecolor="w")

    ax.set_xlim(-w_new, 2 * np.pi + w_new)
    ax.set_ylim(-1.2, 1.2)
    ax.axis("off")

    if phasor:

        hp, hc = polarizations(wts, A=1)

        for i, wt in enumerate(wts):
            ax = fig.add_subplot(gs[2, i])

            ax.axhline(0, 0.5, c="gray", lw=1)
            ax.axvline(0, 0.5, c="gray", lw=1)
            kws = dict(
                fontsize=14, horizontalalignment="right", verticalalignment="top"
            )
            if i == 0:
                ax.annotate(r"$h_+$", xy=(1.45, -0.1), **kws)
                ax.annotate(r"$h_\times$", xy=(-0.05, 1.5), **kws)


            wedge = EllipticalWedge(
                [0, 0],
                1,
                0,
                np.rad2deg(wt),
                color=c[i0],
                alpha=0.1,
            )
            ax.add_patch(wedge)
            
            # Circle on lower subplot
            ax.plot(hp2, hc2, c=c[Npoints // 2])
            # Arrow showing angle
            ax.annotate(
                "",
                xy=(hp[i], hc[i]),
                xytext=(0, 0),
                arrowprops=dict(
                    arrowstyle="->", color=c[-1], lw=2, shrinkA=0, shrinkB=0
                ),
            )

            ax.axis("off")
            ax.axis("equal")
            ax.set_xlim(-1.5, 1.5)
            ax.set_ylim(-1.5, 1.5)
    return fig

In [ ]:
fig = plot_diagram(palette="flare")
save_figure(fig, "ring_plot")